In [1]:
import numpy as np
import json
import pandas as pd

def compute_X_normalized(A, t = -1, lm = 1, save_prefix = ""):
    n, _ = A.shape
    d    = A @ np.ones((n, 1))
    P    = A / d
    Identity = np.identity(A.shape[0])
    e = np.ones((A.shape[0], 1))

    D = np.diag(d.flatten())
    # Compute W
    scale = np.matmul(e.T, np.matmul(D, e))[0, 0]
    W = np.multiply(1 / scale, np.matmul(e, np.matmul(e.T, D)))

    up_P = np.multiply(lm, P - W)
    X_ = Identity - up_P
    X_i = np.linalg.pinv(X_)

    if t > 0:
        LP_t = Identity - np.linalg.matrix_power(up_P, t)
        X_i = np.matmul(X_i, LP_t)

    # Save the unnormalized
    np.save(f"{save_prefix}_U.npy", X_i)

    # Normalize with steady state and save it again
    SS = np.sqrt(np.matmul(D, e))
    SS = 1 / SS
    SS = np.diag(SS.flatten())

    np.save(f"{save_prefix}_N.npy", np.matmul(X_i, SS))
    return

def predict_edges_dsd(training_edges, save_prefix=""):
    # Compute the nodemap
    ps, qs, ws = zip(*training_edges)
    nodes      = set(ps).union(set(qs))
    nodemap    = {k: i for i, k in enumerate(nodes)}
    
    # Save the nodemap
    with open(f"{save_prefix}.json", "w") as jf:
        json.dump(nodemap, jf)
    
    n          = len(nodemap)
    A          = np.zeros((n, n))
    for p, q, w in training_edges:
        A[nodemap[p], nodemap[q]] = w
        A[nodemap[q], nodemap[p]] = w 

    # Compute the embedding and save it
    compute_X_normalized(A, t = -1, lm = 1, save_prefix = save_prefix)

In [4]:
nets = ["y2h-hiconf", "coip-full", "coip-shared", "coip-hiconf", "biotin-full"]
for net in nets:
    print(f"Network: {net}")
    for k in range(5):
        print(f"\tFold: {k}")
        save_prefix = f"../data/networks/{net}/k_{k}_training_training_perc_0.75_test_neg_ratio_20"
            
        # Do some minor filtering
        training_edges = pd.read_csv(f"{save_prefix}.txt", sep = "\t", header = None)
        training_edges.loc[training_edges[2] == 0, [2]] = 0.001
        training_edges = training_edges.values
            
        # Compute embedding and save
        predict_edges_dsd(training_edges, 
                          save_prefix = save_prefix)

Network: y2h-hiconf
	Fold: 0
	Fold: 1
	Fold: 2
	Fold: 3
	Fold: 4
Network: coip-full
	Fold: 0
	Fold: 1
	Fold: 2
	Fold: 3
	Fold: 4
Network: coip-shared
	Fold: 0
	Fold: 1
	Fold: 2
	Fold: 3
	Fold: 4
Network: coip-hiconf
	Fold: 0
	Fold: 1
	Fold: 2
	Fold: 3
	Fold: 4
Network: biotin-full
	Fold: 0
	Fold: 1
	Fold: 2
	Fold: 3
	Fold: 4
